In [1]:
import sys
import sqlite3
from matplotlib.figure import Figure
from matplotlib.backends.backend_qtagg import FigureCanvasQTAgg as FigureCanvas
from PyQt6.QtGui import QFont, QPixmap, QColor
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QPushButton, QVBoxLayout, QWidget, QColorDialog,
    QLineEdit, QLabel, QCheckBox, QFileDialog, QDialog, QComboBox, QScrollArea,
    QFontDialog, QGroupBox, QFormLayout, QSpinBox, QTabWidget, QDateEdit, QTableWidget, QTableWidgetItem, QHBoxLayout, QListWidget, QMessageBox
)
from PyQt6.QtCore import QDate
import cv2
import subprocess

# Initialize the database and required tables
def init_db():
    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS food (
                        id INTEGER PRIMARY KEY,
                        name TEXT NOT NULL,
                        price REAL NOT NULL,
                        inventory INTEGER,
                        profit_type TEXT,
                        profit_value REAL,
                        factors TEXT)''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS sales (
                        id INTEGER PRIMARY KEY,
                        food_id INTEGER,
                        quantity INTEGER,
                        date TEXT,
                        FOREIGN KEY (food_id) REFERENCES food (id))''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS settings (
                        id INTEGER PRIMARY KEY,
                        key TEXT UNIQUE,
                        value TEXT)''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS devices (
                        id INTEGER PRIMARY KEY,
                        name TEXT NOT NULL,
                        type TEXT NOT NULL,
                        method TEXT NOT NULL,
                        value TEXT NOT NULL)''')
    conn.commit()
    conn.close()

class SalesReportWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('گزارشات مالی')
        self.setGeometry(100, 100, 800, 600)

        self.layout = QVBoxLayout()
        self.setLayout(self.layout)

        self.date_from = QDateEdit(self)
        self.date_from.setDate(QDate.currentDate().addMonths(-1))
        self.layout.addWidget(QLabel("از تاریخ:", self))
        self.layout.addWidget(self.date_from)

        self.date_to = QDateEdit(self)
        self.date_to.setDate(QDate.currentDate())
        self.layout.addWidget(QLabel("تا تاریخ:", self))
        self.layout.addWidget(self.date_to)

        generate_report_btn = QPushButton('تولید گزارش', self)
        generate_report_btn.clicked.connect(self.generate_report)
        self.layout.addWidget(generate_report_btn)

        self.report_label = QLabel('', self)
        self.layout.addWidget(self.report_label)

        self.figure = Figure()
        self.canvas = FigureCanvas(self.figure)
        self.layout.addWidget(self.canvas)

    def generate_report(self):
        date_from = self.date_from.date().toString('yyyy-MM-dd')
        date_to = self.date_to.date().toString('yyyy-MM-dd')

        conn = sqlite3.connect('restaurant.db')
        cursor = conn.cursor()
        cursor.execute('''SELECT food.name, SUM(sales.quantity) as total_sold, food.price, food.profit_type, food.profit_value 
                          FROM sales 
                          JOIN food ON sales.food_id = food.id 
                          WHERE sales.date BETWEEN ? AND ?
                          GROUP BY sales.food_id''', (date_from, date_to))
        rows = cursor.fetchall()
        conn.close()

        report_text = "گزارش فروش:\n"
        total_profit = 0
        sales_data = []

        for row in rows:
            name, total_sold, price, profit_type, profit_value = row
            report_text += f"{name}: {total_sold} فروخته شده\n"
            
            if profit_type == "سود ثابت":
                profit = total_sold * profit_value
            elif profit_type == "قیمت تفاضلی":
                profit = total_sold * (price - profit_value)
            elif profit_type == "درصد":
                profit = total_sold * price * (profit_value / 100)
            total_profit += profit

            sales_data.append((name, total_sold, profit))

        report_text += f"\nکل سود: {total_profit}"
        self.report_label.setText(report_text)

        self.figure.clear()
        ax = self.figure.add_subplot(111)
        names = [data[0] for data in sales_data]
        total_sold = [data[1] for data in sales_data]
        profits = [data[2] for data in sales_data]

        ax.bar(names, total_sold, label='Total Sold')
        ax.bar(names, profits, label='Profit', bottom=total_sold)
        ax.legend()
        self.canvas.draw()

class SettingsWindow(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.initUI()
        self.load_settings()

    def initUI(self):
        self.setWindowTitle('تنظیمات')
        self.setGeometry(100, 100, 400, 800)

        self.layout = QVBoxLayout(self)

        self.scroll_area = QScrollArea(self)
        self.scroll_area.setWidgetResizable(True)

        self.scroll_content = QWidget()
        self.scroll_layout = QVBoxLayout(self.scroll_content)
        self.scroll_area.setWidget(self.scroll_content)

        self.layout.addWidget(self.scroll_area)

        # Initialize attributes
        self.bg_color_main = None
        self.bg_image_main = None
        self.font_main = None
        self.button_bg_color_main = None
        self.button_bg_image_main = None
        self.button_font_color_main = None
        self.bg_color_display = None
        self.bg_image_display = None
        self.font_display = None
        self.button_bg_color_display = None
        self.button_bg_image_display = None
        self.button_font_color_display = None

        # Main settings
        self.add_section_label('تنظیمات اصلی')

        self.bg_color_button_main = QPushButton('انتخاب رنگ پس زمینه اصلی', self)
        self.bg_color_button_main.clicked.connect(self.choose_bg_color_main)
        self.scroll_layout.addWidget(self.bg_color_button_main)

        self.bg_image_button_main = QPushButton('انتخاب تصویر پس زمینه اصلی', self)
        self.bg_image_button_main.clicked.connect(self.choose_bg_image_main)
        self.scroll_layout.addWidget(self.bg_image_button_main)

        self.font_button_main = QPushButton('انتخاب فونت اصلی', self)
        self.font_button_main.clicked.connect(self.choose_font_main)
        self.scroll_layout.addWidget(self.font_button_main)

        self.button_bg_color_button_main = QPushButton('انتخاب رنگ پس زمینه دکمه‌های اصلی', self)
        self.button_bg_color_button_main.clicked.connect(self.choose_button_bg_color_main)
        self.scroll_layout.addWidget(self.button_bg_color_button_main)

        self.button_bg_image_button_main = QPushButton('انتخاب تصویر پس زمینه دکمه‌های اصلی', self)
        self.button_bg_image_button_main.clicked.connect(self.choose_button_bg_image_main)
        self.scroll_layout.addWidget(self.button_bg_image_button_main)

        self.button_font_color_button_main = QPushButton('انتخاب رنگ فونت دکمه‌های اصلی', self)
        self.button_font_color_button_main.clicked.connect(self.choose_button_font_color_main)
        self.scroll_layout.addWidget(self.button_font_color_button_main)

        self.window_size_input_main = QLineEdit(self)
        self.scroll_layout.addWidget(QLabel('اندازه پنجره‌های اصلی:', self))
        self.scroll_layout.addWidget(self.window_size_input_main)

        # Display settings
        self.add_section_label('تنظیمات نمایشگر')

        self.bg_color_button_display = QPushButton('انتخاب رنگ پس زمینه نمایشگر', self)
        self.bg_color_button_display.clicked.connect(self.choose_bg_color_display)
        self.scroll_layout.addWidget(self.bg_color_button_display)

        self.bg_image_button_display = QPushButton('انتخاب تصویر پس زمینه نمایشگر', self)
        self.bg_image_button_display.clicked.connect(self.choose_bg_image_display)
        self.scroll_layout.addWidget(self.bg_image_button_display)

        self.font_button_display = QPushButton('انتخاب فونت نمایشگر', self)
        self.font_button_display.clicked.connect(self.choose_font_display)
        self.scroll_layout.addWidget(self.font_button_display)

        self.button_bg_color_button_display = QPushButton('انتخاب رنگ پس زمینه دکمه‌های نمایشگر', self)
        self.button_bg_color_button_display.clicked.connect(self.choose_button_bg_color_display)
        self.scroll_layout.addWidget(self.button_bg_color_button_display)

        self.button_font_color_button_display = QPushButton('انتخاب رنگ فونت دکمه‌های نمایشگر', self)
        self.button_font_color_button_display.clicked.connect(self.choose_button_font_color_display)
        self.scroll_layout.addWidget(self.button_font_color_button_display)

        self.window_size_input_display = QLineEdit(self)
        self.scroll_layout.addWidget(QLabel('اندازه پنجره‌های نمایشگر:', self))
        self.scroll_layout.addWidget(self.window_size_input_display)

        # Printer settings
        self.add_section_label('تنظیمات پرینتر')

        self.add_device_settings('پرینتر', self.scroll_layout)

        # Camera settings
        self.add_section_label('تنظیمات دوربین')

        self.add_device_settings('دوربین', self.scroll_layout)

        # Card reader settings
        self.add_section_label('تنظیمات کارتخوان')

        self.add_device_settings('کارتخوان', self.scroll_layout)

        # Advanced settings
        self.add_section_label('تنظیمات پیشرفته')

        self.payment_link_input = QLineEdit(self)
        self.scroll_layout.addWidget(QLabel('لینک درگاه پرداخت:', self))
        self.scroll_layout.addWidget(self.payment_link_input)

        self.form_active_checkbox = QCheckBox('فعال‌سازی فرم', self)
        self.scroll_layout.addWidget(self.form_active_checkbox)

        save_button = QPushButton('ذخیره', self)
        save_button.clicked.connect(self.save_settings)
        self.scroll_layout.addWidget(save_button)

        apply_button = QPushButton('اعمال', self)
        apply_button.clicked.connect(self.apply_settings)
        self.scroll_layout.addWidget(apply_button)

    def add_section_label(self, text):
        label = QLabel(text, self)
        label.setStyleSheet("font-weight: bold; font-size: 14px;")
        self.scroll_layout.addWidget(label)

    def add_device_settings(self, device_type, layout):
        group_box = QGroupBox(f'تنظیمات {device_type}', self)
        form_layout = QFormLayout()

        method_combobox = QComboBox(self)
        method_combobox.addItems(['اسکن پورت', 'وارد کردن پورت', 'وارد کردن آی‌پی'])
        form_layout.addRow('روش:', method_combobox)

        method_combobox.currentIndexChanged.connect(lambda: self.update_device_settings_visibility(method_combobox, form_layout, device_type))

        self.port_scan_button = QPushButton(f'اسکن پورت‌های {device_type}', self)
        self.port_scan_button.clicked.connect(lambda: self.scan_ports(device_type))
        form_layout.addRow('اسکن پورت:', self.port_scan_button)

        self.manual_port_input = QLineEdit(self)
        form_layout.addRow('پورت دستی:', self.manual_port_input)

        self.ip_input = QLineEdit(self)
        form_layout.addRow('آی‌پی:', self.ip_input)

        self.device_list = QListWidget(self)
        form_layout.addRow('لیست دستگاه‌ها:', self.device_list)

        self.device_name_input = QLineEdit(self)
        form_layout.addRow('نام دستگاه:', self.device_name_input)

        save_device_button = QPushButton('ذخیره دستگاه', self)
        save_device_button.clicked.connect(lambda: self.save_device(device_type))
        form_layout.addRow(save_device_button)

        edit_device_button = QPushButton('ویرایش دستگاه', self)
        edit_device_button.clicked.connect(lambda: self.edit_device(device_type))
        form_layout.addRow(edit_device_button)

        delete_device_button = QPushButton('حذف دستگاه', self)
        delete_device_button.clicked.connect(lambda: self.delete_device(device_type))
        form_layout.addRow(delete_device_button)

        layout.addWidget(group_box)
        group_box.setLayout(form_layout)

        self.update_device_settings_visibility(method_combobox, form_layout, device_type)

    def update_device_settings_visibility(self, method_combobox, form_layout, device_type):
        method = method_combobox.currentText()
        self.port_scan_button.setVisible(method == 'اسکن پورت')
        self.manual_port_input.setVisible(method == 'وارد کردن پورت')
        self.ip_input.setVisible(method == 'وارد کردن آی‌پی')

    def scan_ports(self, device_type):
        self.device_list.clear()
        if device_type == 'دوربین':
            self.scan_cameras()
        elif device_type == 'نمایشگر':
            self.scan_displays()
        elif device_type == 'پرینتر':
            self.scan_printers()
        elif device_type == 'کارتخوان':
            self.scan_card_readers()

    def scan_cameras(self):
        index = 0
        while True:
            cap = cv2.VideoCapture(index)
            if not cap.read()[0]:
                break
            self.device_list.addItem(f'Camera {index}')
            cap.release()
            index += 1

    def scan_displays(self):
        result = subprocess.run(['xrandr', '--listmonitors'], stdout=subprocess.PIPE)
        output = result.stdout.decode('utf-8').split('\n')
        for line in output:
            if 'Monitor' in line:
                self.device_list.addItem(line.split(' ')[-1])

    def scan_printers(self):
        result = subprocess.run(['lpstat', '-p'], stdout=subprocess.PIPE)
        output = result.stdout.decode('utf-8').split('\n')
        for line in output:
            if 'printer' in line:
                self.device_list.addItem(line.split(' ')[1])

    def scan_card_readers(self):
        # Implement card reader scanning logic here
        # For demonstration, we'll just add some dummy data
        self.device_list.addItem('Card Reader 1 - Port 1234')
        self.device_list.addItem('Card Reader 2 - Port 5678')

    def save_device(self, device_type):
        name = self.device_name_input.text()
        method = 'اسکن پورت' if self.port_scan_button.isVisible() else 'وارد کردن پورت' if self.manual_port_input.isVisible() else 'وارد کردن آی‌پی'
        value = self.device_list.currentItem().text() if self.port_scan_button.isVisible() else self.manual_port_input.text() if self.manual_port_input.isVisible() else self.ip_input.text()

        conn = sqlite3.connect('restaurant.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO devices (name, type, method, value) VALUES (?, ?, ?, ?)", (name, device_type, method, value))
        conn.commit()
        conn.close()

        self.load_devices(device_type)

    def edit_device(self, device_type):
        selected_item = self.device_list.currentItem()
        if selected_item:
            name, method_value = selected_item.text().split(' - ')
            method, value = method_value.split(': ')
            self.device_name_input.setText(name)
            if method == 'اسکن پورت':
                self.port_scan_button.setVisible(True)
                self.manual_port_input.setVisible(False)
                self.ip_input.setVisible(False)
            elif method == 'وارد کردن پورت':
                self.port_scan_button.setVisible(False)
                self.manual_port_input.setVisible(True)
                self.ip_input.setVisible(False)
                self.manual_port_input.setText(value)
            elif method == 'وارد کردن آی‌پی':
                self.port_scan_button.setVisible(False)
                self.manual_port_input.setVisible(False)
                self.ip_input.setVisible(True)
                self.ip_input.setText(value)

    def delete_device(self, device_type):
        selected_item = self.device_list.currentItem()
        if selected_item:
            name, method_value = selected_item.text().split(' - ')
            method, value = method_value.split(': ')

            conn = sqlite3.connect('restaurant.db')
            cursor = conn.cursor()
            cursor.execute("DELETE FROM devices WHERE name = ? AND type = ? AND method = ? AND value = ?", (name, device_type, method, value))
            conn.commit()
            conn.close()

            self.load_devices(device_type)

    def load_devices(self, device_type):
        conn = sqlite3.connect('restaurant.db')
        cursor = conn.cursor()
        cursor.execute("SELECT name, method, value FROM devices WHERE type = ?", (device_type,))
        rows = cursor.fetchall()
        conn.close()

        self.device_list.clear()
        for row in rows:
            self.device_list.addItem(f'{row[0]} - {row[1]}: {row[2]}')

    def choose_bg_color_main(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.bg_color_main = color.name()
            self.bg_image_main = None
            self.parent().central_widget.setStyleSheet(f"background-color: {self.bg_color_main};")

    def choose_bg_image_main(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "انتخاب تصویر پس زمینه", "", "Images (*.png *.jpg *.bmp)")
        if file_name:
            self.bg_image_main = file_name
            self.bg_color_main = None
            self.parent().central_widget.setStyleSheet(f"background-image: url({self.bg_image_main});")

    def choose_font_main(self):
        font, ok = QFontDialog.getFont()
        if ok:
            self.font_main = font
            self.parent().setFont(self.font_main)

    def choose_button_bg_color_main(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.button_bg_color_main = color.name()
            self.button_bg_image_main = None
            for i in range(self.parent().central_widget.layout().count()):
                widget = self.parent().central_widget.layout().itemAt(i).widget()
                if isinstance(widget, QPushButton):
                    widget.setStyleSheet(f"background-color: {self.button_bg_color_main};")

    def choose_button_bg_image_main(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "انتخاب تصویر پس زمینه دکمه‌ها", "", "Images (*.png *.jpg *.bmp)")
        if file_name:
            self.button_bg_image_main = file_name
            self.button_bg_color_main = None
            for i in range(self.parent().central_widget.layout().count()):
                widget = self.parent().central_widget.layout().itemAt(i).widget()
                if isinstance(widget, QPushButton):
                    widget.setStyleSheet(f"background-image: url({self.button_bg_image_main});")

    def choose_button_font_color_main(self):

        color = QColorDialog.getColor()
        if color.isValid():
            self.button_font_color_main = color.name()
            for i in range(self.parent().central_widget.layout().count()):
                widget = self.parent().central_widget.layout().itemAt(i).widget()
                if isinstance(widget, QPushButton):
                    widget.setStyleSheet(f"color: {self.button_font_color_main};")

    def choose_bg_color_display(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.bg_color_display = color.name()
            self.bg_image_display = None

    def choose_bg_image_display(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "انتخاب تصویر پس زمینه نمایشگر", "", "Images (*.png *.jpg *.bmp)")
        if file_name:
            self.bg_image_display = file_name
            self.bg_color_display = None

    def choose_font_display(self):
        font, ok = QFontDialog.getFont()
        if ok:
            self.font_display = font

    def choose_button_bg_color_display(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.button_bg_color_display = color.name()
            self.button_bg_image_display = None

    def choose_button_font_color_display(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.button_font_color_display = color.name()

    def save_settings(self):
        conn = sqlite3.connect('restaurant.db')
        cursor = conn.cursor()

        settings = {
            'bg_main': self.bg_color_main if hasattr(self, 'bg_color_main') else self.bg_image_main,
            'font_family_main': self.font_main.family() if hasattr(self, 'font_main') else '',
            'font_size_main': str(self.font_main.pointSize()) if hasattr(self, 'font_main') else '',
            'button_bg_color_main': self.button_bg_color_main if hasattr(self, 'button_bg_color_main') else '',
            'button_bg_image_main': self.button_bg_image_main if hasattr(self, 'button_bg_image_main') else '',
            'button_font_color_main': self.button_font_color_main if hasattr(self, 'button_font_color_main') else '',
            'window_size_main': self.window_size_input_main.text(),
            'bg_display': self.bg_color_display if hasattr(self, 'bg_color_display') else self.bg_image_display,
            'font_family_display': self.font_display.family() if hasattr(self, 'font_display') else '',
            'font_size_display': str(self.font_display.pointSize()) if hasattr(self, 'font_display') else '',
            'button_bg_color_display': self.button_bg_color_display if hasattr(self, 'button_bg_color_display') else '',
            'button_font_color_display': self.button_font_color_display if hasattr(self, 'button_font_color_display') else '',
            'window_size_display': self.window_size_input_display.text(),
            'payment_link': self.payment_link_input.text(),
            'form_active': 'true' if self.form_active_checkbox.isChecked() else 'false'
        }

        for key, value in settings.items():
            cursor.execute("REPLACE INTO settings (key, value) VALUES (?, ?)", (key, value))

        conn.commit()
        conn.close()

    def load_settings(self):
        conn = sqlite3.connect('restaurant.db')
        cursor = conn.cursor()
        cursor.execute("SELECT key, value FROM settings")
        rows = cursor.fetchall()
        conn.close()

        settings = {key: value for key, value in rows}

        if 'bg_main' in settings:
            bg_main = settings['bg_main']
            if bg_main:
                if bg_main.startswith('#'):
                    self.bg_color_main = bg_main
                    self.parent().central_widget.setStyleSheet(f"background-color: {self.bg_color_main};")
                else:
                    self.bg_image_main = bg_main
                    self.parent().central_widget.setStyleSheet(f"background-image: url({self.bg_image_main});")

        if 'font_family_main' in settings or 'font_size_main' in settings:
            self.font_main = QFont()
            if 'font_family_main' in settings:
                self.font_main.setFamily(settings['font_family_main'])
            if 'font_size_main' in settings and settings['font_size_main']:
                self.font_main.setPointSize(int(settings['font_size_main']))
            else:
                self.font_main.setPointSize(12)  # Default value
            self.parent().setFont(self.font_main)

        if 'button_bg_color_main' in settings:
            if settings['button_bg_color_main']:
                self.button_bg_color_main = settings['button_bg_color_main']
                for i in range(self.parent().central_widget.layout().count()):
                    widget = self.parent().central_widget.layout().itemAt(i).widget()
                    if isinstance(widget, QPushButton):
                        widget.setStyleSheet(f"background-color: {self.button_bg_color_main};")

        if 'button_bg_image_main' in settings:
            if settings['button_bg_image_main']:
                self.button_bg_image_main = settings['button_bg_image_main']
                for i in range(self.parent().central_widget.layout().count()):
                    widget = self.parent().central_widget.layout().itemAt(i).widget()
                    if isinstance(widget, QPushButton):
                        widget.setStyleSheet(f"background-image: url({self.button_bg_image_main});")

        if 'button_font_color_main' in settings:
            self.button_font_color_main = settings['button_font_color_main']
            for i in range(self.parent().central_widget.layout().count()):
                widget = self.parent().central_widget.layout().itemAt(i).widget()
                if isinstance(widget, QPushButton):
                    widget.setStyleSheet(f"color: {self.button_font_color_main};")

        self.window_size_input_main.setText(settings.get('window_size_main', ''))

        if 'bg_display' in settings:
            bg_display = settings['bg_display']
            if bg_display:
                if bg_display.startswith('#'):
                    self.bg_color_display = bg_display
                else:
                    self.bg_image_display = bg_display

        if 'font_family_display' in settings or 'font_size_display' in settings:
            self.font_display = QFont()
            if 'font_family_display' in settings:
                self.font_display.setFamily(settings['font_family_display'])
            if 'font_size_display' in settings and settings['font_size_display']:
                self.font_display.setPointSize(int(settings['font_size_display']))
            else:
                self.font_display.setPointSize(12)  # Default value

        if 'button_bg_color_display' in settings:
            if settings['button_bg_color_display']:
                self.button_bg_color_display = settings['button_bg_color_display']

        if 'button_font_color_display' in settings:
            self.button_font_color_display = settings['button_font_color_display']

        self.window_size_input_display.setText(settings.get('window_size_display', ''))

        self.payment_link_input.setText(settings.get('payment_link', ''))
        self.form_active_checkbox.setChecked(settings.get('form_active', 'false') == 'true')

        # Load devices for printer, camera, display, and card reader
        self.load_devices('پرینتر')
        self.load_devices('دوربین')
        self.load_devices('نمایشگر')
        self.load_devices('کارتخوان')

    def apply_settings(self):
        self.save_settings()
        self.parent().load_settings()

class RestaurantManagementSystem(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.load_settings()

    def initUI(self):
        self.setWindowTitle('Restaurant Management System')
        self.setGeometry(100, 100, 400, 300)
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout()
        self.central_widget.setLayout(self.layout)

        btn_manage_food = QPushButton('1- مدیریت غذا', self)
        btn_manage_food.clicked.connect(self.manage_food)
        self.layout.addWidget(btn_manage_food)

        btn_financial_info = QPushButton('2- اطلاعات مالی', self)
        btn_financial_info.clicked.connect(self.financial_info)
        self.layout.addWidget(btn_financial_info)

        btn_settings = QPushButton('3- تنظیمات', self)
        btn_settings.clicked.connect(self.settings)
        self.layout.addWidget(btn_settings)

        btn_order_status = QPushButton('4- وضعیت سفارش', self)
        btn_order_status.clicked.connect(self.order_status)
        self.layout.addWidget(btn_order_status)

        btn_exit = QPushButton('5- خروج از برنامه', self)
        btn_exit.clicked.connect(self.close)
        self.layout.addWidget(btn_exit)

    def manage_food(self):
        self.food_window = FoodManagementWindow()
        self.food_window.show()

    def financial_info(self):
        self.sales_report_window = SalesReportWindow()
        self.sales_report_window.show()

    def settings(self):
        self.settings_window = SettingsWindow(self)
        self.settings_window.exec()

    def order_status(self):
        # Implement order status functionality here
        pass

    def load_settings(self):
        conn = sqlite3.connect('restaurant.db')
        cursor = conn.cursor()
        cursor.execute("SELECT key, value FROM settings")
        rows = cursor.fetchall()
        conn.close()

        settings = {key: value for key, value in rows}

        if 'bg_main' in settings:
            bg_main = settings['bg_main']
            if bg_main:
                if bg_main.startswith('#'):
                    self.central_widget.setStyleSheet(f"background-color: {bg_main};")
                else:
                    self.central_widget.setStyleSheet(f"background-image: url({bg_main});")

        if 'font_color_main' in settings:
            self.setStyleSheet(f"color: {settings['font_color_main']};")

        if 'font_size_main' in settings:
            font = self.font()
            if settings['font_size_main']:
                font.setPointSize(int(settings['font_size_main']))
            else:
                font.setPointSize(12)  # Default value
            self.setFont(font)

        if 'button_bg_color_main' in settings:
            for i in range(self.central_widget.layout().count()):
                widget = self.central_widget.layout().itemAt(i).widget()
                if isinstance(widget, QPushButton):
                    widget.setStyleSheet(f"background-color: {settings['button_bg_color_main']};")

        if 'button_bg_image_main' in settings:
            if settings['button_bg_image_main']:
                for i in range(self.central_widget.layout().count()):
                    widget = self.central_widget.layout().itemAt(i).widget()
                    if isinstance(widget, QPushButton):
                        widget.setStyleSheet(f"background-image: url({settings['button_bg_image_main']});")

        if 'button_font_color_main' in settings:
            for i in range(self.central_widget.layout().count()):
                widget = self.central_widget.layout().itemAt(i).widget()
                if isinstance(widget, QPushButton):
                    widget.setStyleSheet(f"color: {settings['button_font_color_main']};")

        if 'window_size_main' in settings:
            if settings['window_size_main']:
                width, height = map(int, settings['window_size_main'].split('x'))
            else:
                width, height = 800, 600  # Default value
            self.setGeometry(self.x(), self.y(), width, height)

class FoodManagementWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('مدیریت غذا')
        self.setGeometry(100, 100, 600, 400)

        self.layout = QVBoxLayout()
        self.setLayout(self.layout)

        form_layout = QFormLayout()

        self.name_input = QLineEdit(self)
        self.price_input = QLineEdit(self)
        self.inventory_input = QLineEdit(self)
        self.profit_type_input = QComboBox(self)
        self.profit_type_input.addItems(["سود ثابت", "قیمت تفاضلی", "درصد"])
        self.profit_value_input = QLineEdit(self)
        self.factors_layout = QVBoxLayout()

        form_layout.addRow(QLabel("نام غذا:", self), self.name_input)
        form_layout.addRow(QLabel("قیمت غذا:", self), self.price_input)
        form_layout.addRow(QLabel("موجودی:", self), self.inventory_input)
        form_layout.addRow(QLabel("نوع سود:", self), self.profit_type_input)
        form_layout.addRow(QLabel("مقدار سود:", self), self.profit_value_input)
        
        self.layout.addLayout(form_layout)

        add_factor_button = QPushButton('اضافه کردن عامل', self)
        add_factor_button.clicked.connect(self.add_factor)
        self.layout.addWidget(add_factor_button)
        self.layout.addLayout(self.factors_layout)

        save_button = QPushButton('ذخیره', self)
        save_button.clicked.connect(self.save_food)
        self.layout.addWidget(save_button)

        self.table = QTableWidget(self)
        self.layout.addWidget(self.table)

        self.load_data()

    def add_factor(self):
        factor_layout = QHBoxLayout()
        title_input = QLineEdit(self)
        value_input = QLineEdit(self)
        factor_layout.addWidget(QLabel("عنوان:", self))
        factor_layout.addWidget(title_input)
        factor_layout.addWidget(QLabel("میزان هزینه:", self))
        factor_layout.addWidget(value_input)
        self.factors_layout.addLayout(factor_layout)

    def load_data(self):
        conn = sqlite3.connect('restaurant.db')
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM food")
        rows = cursor.fetchall()
        conn.close()

        self.table.setRowCount(len(rows))
        self.table.setColumnCount(6)
        self.table.setHorizontalHeaderLabels(["ID", "نام", "قیمت", "موجودی", "نوع سود", "مقدار سود"])

        for row_idx, row in enumerate(rows):
            for col_idx, item in enumerate(row):
                self.table.setItem(row_idx, col_idx, QTableWidgetItem(str(item)))

    def save_food(self):
        name = self.name_input.text()
        price = float(self.price_input.text())
        inventory = int(self.inventory_input.text())
        profit_type = self.profit_type_input.currentText()
        profit_value = float(self.profit_value_input.text())

        conn = sqlite3.connect('restaurant.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO food (name, price, inventory, profit_type, profit_value) VALUES (?, ?, ?, ?, ?)",
                       (name, price, inventory, profit_type, profit_value))
        conn.commit()
        conn.close()

        self.load_data()

if __name__ == '__main__':
    init_db()
    app = QApplication(sys.argv)
    window = RestaurantManagementSystem()
    window.show()
    sys.exit(app.exec())

SystemExit: 0

C:\Users\ahmac\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
